In [217]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np

In [218]:
# Create the optimization model
question_2_model = gb.Model("Question 2: Baffin Bay")

In [219]:
cost_matrix_df = pd.read_csv(r"C:\Users\gabri\Downloads\cost_matrix.csv")
distributions_df = pd.read_csv(r"C:\Users\gabri\Downloads\distributions.csv")

In [220]:
cost_matrix_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000000,1.988286,1.513698,1.527394,1.841806,2.139559,1.998129,2.278082,1.783403,2.709619,1.788280,1.584201,2.856943,2.521771,1.334300
1,1.988286,0.000000,1.720943,1.339885,1.147908,1.849205,1.372584,2.068157,1.855567,1.301883,1.871727,1.961153,1.429755,1.547382,2.266448
2,1.513698,1.720943,0.000000,2.237808,1.574700,2.658306,1.707801,1.432825,2.155869,2.326255,2.092072,2.417186,2.380657,2.155311,1.609137
3,1.527394,1.339885,2.237808,0.000000,1.821132,1.901807,1.995907,1.926461,1.288935,2.031147,1.901646,2.217846,2.382201,2.348028,1.512629
4,1.841806,1.147908,1.574700,1.821132,0.000000,2.586485,2.147243,1.553036,1.740977,2.272306,1.174655,1.802144,2.402317,1.854828,2.365839
5,2.139559,1.849205,2.658306,1.901807,2.586485,0.000000,2.001719,2.696914,1.918241,1.851361,1.898910,1.903692,1.768955,2.925275,1.165120
6,1.998129,1.372584,1.707801,1.995907,2.147243,2.001719,0.000000,1.653432,1.520055,2.338334,1.516578,2.213800,2.341547,2.009148,1.478275
7,2.278082,2.068157,1.432825,1.926461,1.553036,2.696914,1.653432,0.000000,1.968242,2.190289,2.319021,2.611285,2.698317,1.317032,1.682872
8,1.783403,1.855567,2.155869,1.288935,1.740977,1.918241,1.520055,1.968242,0.000000,1.188825,2.361799,2.255721,1.992090,1.850866,2.055041
9,2.709619,1.301883,2.326255,2.031147,2.272306,1.851361,2.338334,2.190289,1.188825,0.000000,2.034125,1.599529,2.754371,2.012707,1.989721


In [221]:
distributions_df

,Distribution,n,p,Mean,Python Function Call
0,Binomial,119,0.755413,89.894195,np.random.binomial
1,Binomial,88,0.588828,51.816857,np.random.binomial
2,Binomial,139,0.712967,99.102430,np.random.binomial
3,Binomial,115,0.580407,66.746849,np.random.binomial
4,Binomial,82,0.651873,53.453597,np.random.binomial
5,Binomial,98,0.630353,61.774597,np.random.binomial
6,Binomial,130,0.881940,114.652169,np.random.binomial
7,Binomial,124,0.887382,110.035412,np.random.binomial
8,Binomial,102,0.976313,99.583887,np.random.binomial
9,Binomial,117,0.840305,98.315728,np.random.binomial


In [222]:
n = distributions_df['n'].values
p = distributions_df['p'].values
mean_values = distributions_df['Mean'].values

a)

The first cost: The unsold inventory cost at the end of period two per unit is $24.44  

The second cost: The opportunity cost of excess inventory left over at the end of period two is 49.99 - 24.44 = $25.55

b)

In [223]:
# Define constants for costs from part (a)
unsold_inventory_cost = 24.44
opportunity_cost_per_unit = 49.99 - 24.44  # Selling price - unit ordering cost

# Initialize variables for total cost across all trials
total_cost = 0

# Set up Monte Carlo simulation
num_trials = 50
num_scenarios_per_trial = 100

for trial in range(num_trials):
    # Generate demand scenarios
    demand_scenarios = np.random.binomial(n=n.astype('int32'), p=p.astype('float32'), size=(num_scenarios_per_trial, len(n)))

    # Initialize model
    model = gb.Model("Optimal Order Quantity")

    # Decision variables: order quantity for each store
    y = model.addVars(len(n), lb=0, vtype=GRB.INTEGER, name="Order Quantity")

    # Objective function: minimize total cost
    total_cost_var = gb.quicksum(unsold_inventory_cost * y[i] for i in range(len(n))) + gb.quicksum((y[i] - demand_scenarios[j, i]) * opportunity_cost_per_unit for i in range(len(n)) for j in range(num_scenarios_per_trial))
    model.setObjective(total_cost_var, GRB.MINIMIZE)

    # Constraints: demand satisfaction for each store
    for i in range(len(n)):
        for j in range(num_scenarios_per_trial):
            model.addConstr(y[i] >= demand_scenarios[j, i], f"Demand_{i}_{j}")

    # Solve the model
    model.optimize()

    # Update total cost
    total_cost += model.objVal

# Calculate average cost
average_cost = total_cost / num_trials
print("Average cost associated with the optimal plan:", average_cost)
print("Objective Function:", model.objVal)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1500 rows, 15 columns and 1500 nonzeros
Model fingerprint: 0xba769c96
Variable types: 0 continuous, 15 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+03, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+02]
Found heuristic solution: objective 415639.84000
Presolve removed 1500 rows and 15 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 415640 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.156398400000e+05, best bound 4.156398400000e+05, gap 0.0000%
Gurobi Optimizer version

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1500 rows, 15 columns and 1500 nonzeros
Model fingerprint: 0xfd95e377
Variable types: 0 continuous, 15 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+03, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+02]
Found heuristic solution: objective 411015.29000
Presolve removed 1500 rows and 15 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 411015 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.110152900000e+05, best bound 4.110152900000e+05, gap 0.0000%
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core

Part c)

In [244]:
# Define constants for costs
unsold_inventory_cost = 24.44
opportunity_cost_per_unit = 49.99 - 24.44  # Selling price - unit ordering cost

# Initialize variables for total cost across all trials
total_cost = 0

# Set up Monte Carlo simulation
num_trials = 50
num_scenarios_per_trial = 100

for trial in range(num_trials):
    # Generate demand scenarios
    demand_scenarios = np.random.binomial(n=n.astype('int32'), p=p.astype('float32'), size=(num_scenarios_per_trial, len(n)))

    # Initialize model
    model = gb.Model("Optimal Order Quantity with Transshipment")

    # Decision variables: order quantity for each store
    y = model.addVars(len(n), lb=0, vtype=GRB.INTEGER, name="OrderQuantity")

    # Transshipment decision variables: flow from store i to store j
    x = model.addVars(len(n), len(n), lb=0, vtype=GRB.INTEGER, name="Transshipment")

    # Objective function: minimize total cost
    total_cost_var = gb.quicksum(unsold_inventory_cost * y[i] for i in range(len(n)))

    # Add opportunity cost for each scenario
    for j in range(num_scenarios_per_trial):
        total_cost_var += gb.quicksum((y[i] - demand_scenarios[j, i]) * opportunity_cost_per_unit for i in range(len(n)))

    # Add transshipment cost
    total_cost_var += gb.quicksum(cost_matrix_df.iloc[i, j] * x[i, j] for i in range(len(n)) for j in range(len(n)))

    model.setObjective(total_cost_var, GRB.MINIMIZE)

    for i in range(len(n)):
        model.addConstr(y[i] >= gb.quicksum(x[i,k] for k in range(len(n)) if i !=k), f"Demand_{i}_{j}")

    # Constraints: demand satisfaction for each store
    for i in range(len(n)):
        for j in range(num_scenarios_per_trial):
            model.addConstr((y[i] + gb.quicksum(x[i,k] for k in range(len(n)) if i !=k)) >= demand_scenarios[j, i], f"Demand_{i}_{j}")

    # Constraints: transshipment flow balance
    for i in range(len(n)):
        inflow = x.sum(i, '*')  # sum of inflows to store i
        outflow = x.sum('*', i)  # sum of outflows from store i
        model.addConstr(inflow - outflow == 0, f"FlowBalance_{i}")

    # Constraints: flow cannot exceed available inventory
    for i in range(len(n)):
        for j in range(len(n)):
            if i != j:
                model.addConstr(x[i, j] <= y[i], f"InventoryLimit_{i}_{j}")
    
    # Constraints: flow from a store to itself should be zero
    for i in range(len(n)):
        model.addConstr(x[i, i] == 0, f"NoSelfTransshipment_{i}")

    for i in range(len(n)):
        for j in range(len(n)):
            model.addConstr(x[i, j] == x[j,i], f"Transshipment_{i}")

    # Modify Transshipment Constraints to encourage satisfying demand
    for i in range(len(n)):
        for k in range(num_scenarios_per_trial):  
            # Adjust transshipment to only occur when it helps satisfy demand
            model.addConstr(gb.quicksum(x[i, j] for j in range(len(n)) if i != j) >= demand_scenarios[k, i] - y[i], f"Transshipment_{i}_{k}")

    # Solve the model
    model.optimize()

    # Update total cost
    total_cost += model.objVal

# Calculate average cost
average_cost = total_cost / num_trials
print("Average cost associated with the optimal plan:", average_cost)
print("Objective Function:", model.objVal)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3480 rows, 240 columns and 46500 nonzeros
Model fingerprint: 0x2f8a9e5a
Variable types: 0 continuous, 240 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+02]
Found heuristic solution: objective 403939.05000
Presolve removed 3450 rows and 120 columns
Presolve time: 0.01s
Presolved: 30 rows, 120 columns, 450 nonzeros
Variable types: 0 continuous, 120 integer (0 binary)
Found heuristic solution: objective 365278.32516

Root relaxation: objective -1.555351e+06, 16 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  

In [245]:
print(model.printAttr('X'))


    Variable            X 
-------------------------
OrderQuantity[0]           50 
OrderQuantity[1]           32 
OrderQuantity[2]           56 
OrderQuantity[3]           40 
OrderQuantity[4]           32 
OrderQuantity[5]           36 
OrderQuantity[6]           62 
OrderQuantity[7]           59 
OrderQuantity[8]           51 
OrderQuantity[9]           55 
OrderQuantity[10]           46 
OrderQuantity[11]           53 
OrderQuantity[12]           61 
OrderQuantity[13]           65 
OrderQuantity[14]           63 
Transshipment[0,2]           29 
Transshipment[0,3]           13 
Transshipment[0,11]            8 
Transshipment[1,6]            1 
Transshipment[1,9]            1 
Transshipment[1,12]           30 
Transshipment[2,0]           29 
Transshipment[2,4]            1 
Transshipment[2,6]           26 
Transshipment[3,0]           13 
Transshipment[3,8]           26 
Transshipment[3,12]            1 
Transshipment[4,2]            1 
Transshipment[4,10]           31 
Transshipm